# DEBUG: Inference performance comparison                                                                                                             
#  TFLite - Performance CPU only vs. Heterogenous execution
In this example notebook, we compare ***TFlite*** inference performance, of a pre-trained Classification model running on CPU (Cortex A**) vs. the same model running in an heterogenous approach (Cortex A** + TIDL offload)

   - The user can choose the model (see section titled *Choosing a Pre-Compiled Model*)
   - The models used in this example were trained on the ***ImageNet*** dataset because it is a widely used dataset developed for training and benchmarking image classification AI models. 
   - We perform inference on one sample image.
   
## Choosing a Pre-Compiled Model
We provide a set of precompiled artifacts to use with this notebook that will appear as a drop-down list once the first code cell is executed.

<img src=docs/images/drop_down.PNG width="400">

**Note**:Users can run this notebook as-is, only action required is to select a model. 

In [ ]:
import os
import cv2
import numpy as np
import ipywidgets as widgets
from scripts.utils import get_eval_configs
last_artifacts_id = selected_model_id.value if "selected_model_id" in locals() else None
prebuilt_configs, selected_model_id = get_eval_configs('classification','tflitert', num_quant_bits = 8, last_artifacts_id = last_artifacts_id)
display(selected_model_id)

In [ ]:
print(f'Selected Model: {selected_model_id.label}')
config = prebuilt_configs[selected_model_id.value]
config['session'].set_param('model_id', selected_model_id.value)
config['session'].start()

## Define utility function to preprocess input images

Below, we define a utility function to preprocess images for the model. This function takes a path as input, loads the image and preprocesses the images as required by the model. The steps below are shown as a reference (no user action required):

 1. Load image
 2. Convert BGR image to RGB
 3. Scale image
 4. Apply per-channel pixel scaling and mean subtraction
 5. Convert RGB Image to BGR. 
 6. Convert the image to NCHW format


- The input arguments of this utility function is selected automatically by this notebook based on the model selected in the drop-down

In [ ]:
def preprocess(image_path, size, mean, scale, layout, reverse_channels):
    # Step 1
    img = cv2.imread(image_path)
    
    # Step 2
    img = img[:,:,::-1]
    
    # Step 3
    img = cv2.resize(img, (size, size), interpolation=cv2.INTER_CUBIC)
     
    # Step 4
    img = img.astype('float32')
    if mean is not None and scale is not None:
        for mean, scale, ch in zip(mean, scale, range(img.shape[2])):
            img[:,:,ch] = ((img.astype('float32')[:,:,ch] - mean) * scale)
    # Step 5
    if reverse_channels:
        img = img[:,:,::-1]
        
    # Step 6
    if layout == 'NCHW':
        img = np.expand_dims(np.transpose(img, (2,0,1)),axis=0)
    else:
        img = np.expand_dims(img,axis=0)
    
    return img

In [ ]:
from scripts.utils import get_preproc_props

size, mean, scale, layout, reverse_channels = get_preproc_props(config)    
print(f'Image size: {size}')

## Load and Run a model on ARM (Cortex A**) only

Next cell executes ***TF Lite*** model on Cortex A, and collect benchmark data.

In [ ]:
import tflite_runtime.interpreter as tflite
import matplotlib.pyplot as plt
from scripts.utils import imagenet_class_to_name

tflite_model_path = config['session'].get_param('model_file')
artifacts_dir = config['session'].get_param('artifacts_folder')

interpreter = tflite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

img_in = preprocess('sample-images/elephant.bmp' , size, mean, scale, layout, reverse_channels) 

if not input_details[0]['dtype'] == np.float32:
    img_in = np.uint8(img_in)
    
interpreter.set_tensor(input_details[0]['index'], img_in)

#Running inference several times to get an stable performance output
for i in range(5):
    interpreter.invoke()
    
res = interpreter.get_tensor(output_details[0]['index'])

print(f'\nTop three results:')
for idx, cls in enumerate(res[0].squeeze()[1:].argsort()[-3:][::-1]):
    print('[%d] %s' % (idx, '/'.join(imagenet_class_to_name(cls))))
    
from scripts.utils import plot_TI_performance_data, plot_TI_DDRBW_data, get_benchmark_output
print(f'\nPerformance CPU EP')
stats = interpreter.get_TI_benchmark_data()
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10,5))
plot_TI_performance_data(stats, axis=ax)
plt.show()

tt, st, rb, wb = get_benchmark_output(stats)
print(f'Statistics : \n Inferences Per Second   : {1000.0/tt :7.2f} fps')
print(f' Inferece Time Per Image : {tt :7.2f} ms  \n DDR BW Per Image        : {rb+ wb : 7.2f} MB')

## Load and Run a model on Heterogenous mode 

Next cell executes ***TF Lite*** model in heterogenous mode. Model runs on Cortex A** with graphs offload to TIDL using ***`libtidl_tfl_delegate`*** delegate library. Benchmark data is shown at the end.

In [ ]:
tflite_model_path = config['session'].get_param('model_file')
artifacts_dir = config['session'].get_param('artifacts_folder')

tidl_delegate = [tflite.load_delegate('libtidl_tfl_delegate.so', {'artifacts_folder': artifacts_dir})]

interpreter = tflite.Interpreter(model_path=tflite_model_path, experimental_delegates=tidl_delegate)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

img_in = preprocess('sample-images/elephant.bmp' , size, mean, scale, layout, reverse_channels) 

if not input_details[0]['dtype'] == np.float32:
    img_in = np.uint8(img_in)
    
interpreter.set_tensor(input_details[0]['index'], img_in)

#Running inference several times to get an stable performance output
for i in range(5):
    interpreter.invoke()
    
res = interpreter.get_tensor(output_details[0]['index'])

print(f'\nTop three results:')
for idx, cls in enumerate(res[0].squeeze()[1:].argsort()[-3:][::-1]):
    print('[%d] %s' % (idx, '/'.join(imagenet_class_to_name(cls))))
    
from scripts.utils import plot_TI_performance_data, plot_TI_DDRBW_data, get_benchmark_output
print(f'\nPerformance TFLite + TIDL delegates')
stats = interpreter.get_TI_benchmark_data()
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10,5))
plot_TI_performance_data(stats, axis=ax)
plt.show()

tt, st, rb, wb = get_benchmark_output(stats)
print(f'Statistics : \n Inferences Per Second   : {1000.0/tt :7.2f} fps')
print(f' Inferece Time Per Image : {tt :7.2f} ms  \n DDR BW Per Image        : {rb+ wb : 7.2f} MB')

## Final notes

- With this notebook, user's can quickly compare FPS when running their models only on Cortex A** vs. running their models in heterogenous mode.
- If in Heterogenous mode a model's accuracy, or output, is wrong, a quick sanity check is to run the same model only on Cortex A**
- Accuracy can be improved by modifying TIDL compilation options. For additional tips you can check "run and compare a model compiled with different compilation option" inside debug_tips notebook